In [ ]:
 

import pandas as pd 



from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

from IPython.display import display, HTML



from matplotlib import pyplot as plt
import matplotlib.patches as mpatches

import seaborn as sns
%matplotlib inline

In [ ]:
vg_df = pd.read_csv('../input/Video_Games_Sales_as_at_22_Dec_2016.csv')
vg_df.User_Score = vg_df.User_Score.convert_objects(convert_numeric=True)

## Correlation

In [ ]:
plt.figure(figsize=(12, 8))

vg_corr = vg_df.corr()
sns.heatmap(vg_corr, 
            xticklabels = vg_corr.columns.values,
            yticklabels = vg_corr.columns.values,
            annot = True);

## Crossplots

In [ ]:
plt.figure(figsize=(14, 14))

sns.pairplot(vg_df, diag_kind='kde');

# Evolution Over Time

## Sales vs Number of Releases

In [ ]:
#Group the entries by year, then get how many entries are there; i.e. the number of releases
temp1 = vg_df.groupby(['Year_of_Release']).count()
temp1 = temp1.reset_index()

#Do the same, but sum the values to get the total values of everything by year.
temp2 = vg_df.groupby(['Year_of_Release']).sum()
temp2 = temp2.reset_index()

#Normalize the data, i.e. zero mean and unit std. I did this to be able to compare the shapes of both graphs, since 
#they have different ranges
normalised_df = pd.DataFrame()

normalised_df['release_count'] = temp1['Name']
normalised_df['global_sales'] = temp2['Global_Sales']
normalised_df = (normalised_df - normalised_df.mean()) / normalised_df.std()#(normalised_df.max() - normalised_df.min()) 
normalised_df['year'] = temp1['Year_of_Release']


#Plot
plt.figure(figsize=(15, 9))
ax = sns.pointplot(x = normalised_df.year, y = normalised_df.release_count, color = 'blue', label='Release Count')
ax = sns.pointplot(x = normalised_df.year, y = normalised_df.global_sales, color = 'red', label='Global Sales')

blue_patch = mpatches.Patch(color='blue', label='NUMBER OF RELEASES')
red_patch = mpatches.Patch(color='red', label='GLOBAL SALES')
plt.legend(handles=[blue_patch, red_patch], loc='upper left', fontsize = 16)

plt.xticks(rotation=45);

## Genre Sales Evolution

In [ ]:
fig = plt.figure(figsize=(10, 8))

genre_sales_percentages_by_year = (vg_df.groupby(['Year_of_Release', 'Genre']).Global_Sales.sum())*(100)/vg_df.groupby(['Year_of_Release']).Global_Sales.sum()
genre_sales_percentages_by_year.unstack().plot(kind='area',stacked=True, colormap= 'Spectral', grid=False, figsize=(13, 4))

yearlySales = vg_df.groupby(['Year_of_Release','Genre']).Global_Sales.sum()
yearlySales.unstack().plot(kind='area',stacked=True, colormap= 'Spectral', figsize=(13, 4) ) ;

## Genre Total Sales

In [ ]:
x = vg_df.groupby(['Genre']).sum().copy()
ax = x.Global_Sales.sort_values(ascending=False).plot(kind='bar', figsize=(13, 5));

for p in ax.patches:
    ax.annotate(str( round( p.get_height() ) ) + "\n" + str(round( p.get_height() /89.170) )+ "%", 
                (p.get_x() * 1.007, p.get_height() * 0.75),
                color='black')

In [ ]:

genre_difference_metric = [vg_df.Genre.value_counts().index, vg_df.groupby(['Genre']).sum().Global_Sales.sort_values(ascending=False).index, vg_df.groupby(['Genre']).mean().Global_Sales.sort_values(ascending=False).index]


genre_evolution_df = pd.DataFrame(columns=['genre', 'rank_type', 'rank'])

#Populate the dataframe
for metric in range(3):
    for genre in range(len(genre_difference_metric[metric])):
        genre_evolution_df = genre_evolution_df.append({'genre':genre_difference_metric[metric][genre], 'rank_type': metric, 'rank':genre},
                                   ignore_index=True)

        
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)

sns.pointplot(x=genre_evolution_df.rank_type,
              y=12-genre_evolution_df['rank'], 
              hue=genre_evolution_df.genre)

for i in range(len(genre_difference_metric[0])):
    ax.text(-0.75, 12-i, genre_difference_metric[0][i], fontsize=11)
    ax.text(2.1, 12-i, genre_difference_metric[2][i], fontsize=11)
    
ax.set_xlim([-2,4])

xs = [0.0, 1.0, 2.0]
x_labels = ['total releases', 'total sales', 'average sales']
plt.xticks(xs, x_labels, rotation='vertical')

ax.set_xlabel('Sales Metric')

ys = range(1,13)
y_labels = ['12th', '11th', '10th', '9th', '8th', '7th', '6th', '5th', '4th', '3rd', '2nd', '1st']
plt.yticks(ys, y_labels)
ax.set_ylabel('Genre Rank')

plt.show();

## Rating Sales Evolution

In [ ]:
rating_sales_percentages_by_year = (vg_df.groupby(['Year_of_Release', 'Rating']).Global_Sales.sum())*(100)/vg_df.groupby(['Year_of_Release']).Global_Sales.sum()
rating_sales_percentages_by_year.unstack().plot(kind='area',stacked=True, colormap= 'Spectral', figsize=(13, 4));

# Scores

## Critic vs User Scores

In [ ]:
g = sns.jointplot(x = 'Critic_Score', 
              y = 'User_Score',
              data = vg_df, 
              kind = 'hex', 
              cmap= 'hot', 
              size=6)

#http://stackoverflow.com/questions/33288830/how-to-plot-regression-line-on-hexbins-with-seaborn
sns.regplot(vg_df.Critic_Score, vg_df.User_Score, ax=g.ax_joint, scatter=False, color='grey');

# Regional Sales

## Scatterplot between Global Sales and Regional Sales

In [ ]:
sales_cols = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
sales_normalised_df = vg_df[sales_cols].apply(lambda x: (x - x.mean()) / (x.max() - x.min()))

sns.regplot(x = sales_normalised_df.Global_Sales, y = sales_normalised_df.NA_Sales,    marker="+")
sns.regplot(x = sales_normalised_df.Global_Sales, y = sales_normalised_df.EU_Sales,    marker=".")
sns.regplot(x = sales_normalised_df.Global_Sales, y = sales_normalised_df.JP_Sales,    marker="x")
sns.regplot(x = sales_normalised_df.Global_Sales, y = sales_normalised_df.Other_Sales, marker="o")

plt.xlim(-0.05, 1.05)
plt.ylim(-0.05, 1.05)
fig.tight_layout();

## Regional Sales of Genres

In [ ]:
genre_geo_rankings = [vg_df.groupby('Genre').sum().unstack().NA_Sales.sort_values(ascending=False).index, 
                      vg_df.groupby('Genre').sum().unstack().EU_Sales.sort_values(ascending=False).index,
                      vg_df.groupby('Genre').sum().unstack().Other_Sales.sort_values(ascending=False).index,
                      vg_df.groupby('Genre').sum().unstack().JP_Sales.sort_values(ascending=False).index
                      ]

genre_geo_rank_df = pd.DataFrame(columns=['genre', 'rank_type', 'rank'])


for region in range(4):
    for genre in range(len(genre_geo_rankings[region])):
        genre_geo_rank_df = genre_geo_rank_df.append({'genre':genre_geo_rankings[region][genre], 'rank_type': region, 'rank':genre},
                                   ignore_index=True)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)

sns.pointplot(x=genre_geo_rank_df.rank_type,
              y=12-genre_geo_rank_df['rank'], 
              hue=genre_geo_rank_df.genre)

for i in range(len(genre_geo_rankings[0])):
    ax.text(-0.9, 12-i, genre_geo_rankings[0][i], fontsize=11)
    ax.text(3.2, 12-i, genre_geo_rankings[3][i], fontsize=11)
    
ax.set_xlim([-2,5])

xs = [0.0, 1.0, 2.0, 3.0]
x_labels = ['North America', 'E.U.', 'Rest of the World', 'Japan']
plt.xticks(xs, x_labels, rotation='vertical')
ax.set_xlabel('Region')

ys = range(1,13)
y_labels = ['12th', '11th', '10th', '9th', '8th', '7th', '6th', '5th', '4th', '3rd', '2nd', '1st']
plt.yticks(ys, y_labels)
ax.set_ylabel('Genre Rank')

plt.show();

I think there are a lot of insights from this graph. Mainly, the taste of gamers in Japan is really different than that of the three other regions. The remaining three regions are, more or less, similar; with maybe only platform and racing genres moving more than one position between the three regions.

# Most Selling Platform Each Year

I will plot the top selling platform timeline. In the beginning the plot had all the platforms sales, but the result had too much information to digest, it was too clutered. The graph is followed by a table that describes the same thing, just for convenience

In [ ]:
#temp is the sum of all variables for each platform by year
temp = vg_df.groupby(['Year_of_Release', 'Platform']).sum().reset_index().groupby('Year_of_Release')

platform_yearly_winner_df = pd.DataFrame()

for year, group in temp:
    current_year = temp.get_group(year)
    this_year_max_sales = 0.0
    current_year_winner = ""
    row = {'year':"", 'winner':"", 'sales':""}
    for index, platform_data in current_year.iterrows():
        if platform_data.Global_Sales > this_year_max_sales:
            this_year_max_sales = platform_data.Global_Sales
            current_year_winner = platform_data.Platform
    
    row['year'] = year
    row['winner'] = current_year_winner
    row['sales'] = this_year_max_sales
    platform_yearly_winner_df = platform_yearly_winner_df.append(row, ignore_index=True)

fig = plt.figure(figsize=(13, 4))

g = sns.pointplot(x = platform_yearly_winner_df.year ,
              y = platform_yearly_winner_df.sales , 
              hue = platform_yearly_winner_df.winner);

#http://stackoverflow.com/questions/26540035/rotate-label-text-in-seaborn-factorplot
g.set_xticklabels(g.get_xticklabels(), rotation=90);

In [ ]:
platform_yearly_winner_df.set_index('year', inplace=True)
HTML(platform_yearly_winner_df.to_html())